<a href="https://colab.research.google.com/github/svz55/AuthorsBooks/blob/master/chatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Установка библиотек. Сервисные функции
!pip -q install --upgrade tiktoken
!pip -q install langchain openai chromadb
!pip -q install gspread oauth2client

from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re

import os
import openai
import tiktoken
import re


class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class GPT():
  def __init__(self):
    pass

  @classmethod
  def set_key(cls):
      password_input = widgets.Password(
          description='Введите пароль:',
          layout=widgets.Layout(width='500px'),
          style={'description_width': 'initial', 'white-space': 'pre-wrap', 'overflow': 'auto'})
      login_button = widgets.Button(description='Авторизация')
      output = widgets.Output()

      def on_button_clicked(_):
          with output:
              openai.api_key = password_input.value
              os.environ["OPENAI_API_KEY"] = openai.api_key
              print(f'{bcolors.OKGREEN}{bcolors.BOLD}Ключ сохранен!{bcolors.ENDC}')
              password_input.layout.display = 'none'
              login_button.layout.display = 'none'

      login_button.on_click(on_button_clicked)
      display(widgets.VBox([password_input, login_button, output]))

  def load_search_indexes(self, url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return self.create_embedding(text)

  def load_prompt(self,
                  url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return f'{text}'


  def create_embedding(self, data):
    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    source_chunks = []
    splitter = CharacterTextSplitter(separator="\n", chunk_size=1024, chunk_overlap=0)

    for chunk in splitter.split_text(data):
      source_chunks.append(Document(page_content=chunk, metadata={}))

    # Создание индексов документа
    search_index = Chroma.from_documents(source_chunks, OpenAIEmbeddings(), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    print('\n ===========================================: ')
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', 0.0004*(count_token/1000), ' $')
    return search_index

  def answer(self, system, topic, temp = 1):
    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": topic}
      ]

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=temp
      )

    return completion.choices[0].message.content

  def num_tokens_from_messages(self, messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")

  def insert_newlines(self, text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)


  def dialog(self):
    user = ''
    dialog = ''

    print(f'{bcolors.OKBLUE}{bcolors.BOLD}С чем связан ваш интерес к искусственному интеллекту?{bcolors.ENDC}')

    while user.lower() not in ['stop', 'exit', 'выход']:
      user = input('Клиент: ')
      if (user == 'Stop'): break

      dialog += '\n\n' + 'Клиент: ' + user
      add_dialog = gpt.answer(expert_prompt, user)

      dialog += '\n\n' + 'Менеджер: ' + add_dialog
      print(f'\n{bcolors.OKBLUE}{bcolors.BOLD}Менеджер:{bcolors.ENDC} {self.insert_newlines(add_dialog)}')
      report = gpt.answer(validation_prompt, dialog)
      answer = gpt.answer(action_prompt, report)

      print(f'\n{bcolors.OKGREEN}{bcolors.BOLD}Отчёт системы:\n {bcolors.ENDC}{report}')
      print(f'\n{bcolors.HEADER}{bcolors.BOLD}Менеджер: {bcolors.ENDC}{self.insert_newlines(answer)}\n\n')

    return dialog

  def answer_index(self, system, topic, search_index, temp = 1, verbose = 0):

    #Выборка документов по схожести с вопросом
    docs = search_index.similarity_search(topic, k=5)
    if (verbose): print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if (verbose): print('message_content :\n ======================================== \n', message_content)

    messages = [
      {"role": "system", "content": system + f"{message_content}"},
      {"role": "user", "content": topic}
      ]

    # example token count from the function defined above
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f"{self.num_tokens_from_messages(messages, 'gpt-3.5-turbo-0301')} токенов использовано на вопрос")

    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=temp
    )
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    if (verbose): print('\n ===========================================: ')
    if (verbose): print('ЦЕНА запроса с ответом :', 0.002*(completion["usage"]["total_tokens"]/1000), ' $')
    if (verbose): print('\n ===========================================: ')
    print('ОТВЕТ : \n', self.insert_newlines(completion.choices[0].message.content))

    # return completion

  def get_chatgpt_ansver3(self, system, topic, search_index, temp = 1):

    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": topic}
      ]

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=temp
      )
    print('ОТВЕТ : \n', self.insert_newlines(completion.choices[0].message.content))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.3/645.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 388.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.8/927.8 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
gpt = GPT()

In [ ]:
GPT.set_key()

<h3><b> Search_index </b></h3>

In [ ]:
#Последние презентации с вебинаров
search_index = gpt.load_search_indexes('https://docs.google.com/document/d/1Rqn4T0kIiLtJ-eutRZhFsAQ4SPvNcMFKLVDh5KHmMQg/edit?usp=sharing')

In [ ]:
chat_manager_system = 'Ты менеджер поддержки в чате компании, компания продает курсы по AI. У тебя есть большой документ со всеми материалами о продуктах компании. Тебе задает вопрос клиент в чате, дай ему ответ, опираясь на документ, постарайся ответить так, чтобы человек захотел после ответа купить обучение и отчай максимально точно по документу, не придумывай ничего от себя. Документ с информацией для ответа клиенту: '

In [ ]:
gpt.answer_index(
    chat_manager_system,
    'Расскажи про тариф Продвинутый',
    search_index
)

<h3><b> Загрузка prompt'ов</b></h3>

In [ ]:
valid_url = 'https://docs.google.com/document/d/1DkwfJ42FG3ArJJdSR3nvYeOTVPws-mf3jdU0diJzh4Q/edit?usp=sharing'
action_url = 'https://docs.google.com/document/d/1N8dN-xGbvODoMYwOk8Hq7Va2w7KFkh_vIBmIc0AXyYA/edit?usp=sharing'
expert_url = 'https://docs.google.com/document/d/1QhbJlWcJ1KoJUhIAYfLdKNYajoFXKlSzspCAyrOaNYM/edit?usp=sharing'

In [ ]:
valid_text = 'Ты менеджер оценки диалогов менеджеров по продажам с клиентами, компания продает курсы по AI. Тебе будут присылать диалоги, твоя задача создавать по ним отчеты. У тебя есть инструкция, постарайся отвечать максимально точно по инструкции, не придумывай ничего от себя.'
validation_prompt =  valid_text + 'Инструкция \n\n' + gpt.load_prompt(valid_url)

action_text = 'Ты менеджер по продажам, компания продает курсы по AI. Тебе присылают отчет по диалогу с клиентом, который вел другой менеджер по продажам. В отчете информация о том, какие потребности клиента нам известны и какие возражения он говорил. Тебе надо задать клиенту следующий вопрос с целью больше выявить потребность и продать обучение. У тебя есть инструкция, как задавать вопросы клиенту в зависимости от отчета. Постарайся подобрать такие вопросы, чтобы клиент в итоге купил обучение по AI. Отвечай максимально точно по инструкции, не придумывай ничего от себя.'
action_prompt = action_text + 'Инструкция \n\n' +  gpt.load_prompt(action_url)

expert_text = 'Ты менеджер по продажам, компания продает курсы по AI. Тебе будут писать вопросы клиенты. У тебя есть инструкция, как отвечать на возражения клиентов и какие аргументы приводить, если клиент выразил потребность. Постарайся отвечать так, чтобы клиент в итоге купил обучение по AI. Отвечай максимально точно по инструкции, не придумывай ничего от себя'
expert_prompt = expert_text + 'Инструкция \n\n' +  gpt.load_prompt(expert_url)

<h3><b> Одиночный запрос по prompt'у</h3>

In [ ]:
gpt.answer(expert_prompt,
           'Я подумываю о том, чтобы сменить профессию на AI разработчика')

'Отлично, это замечательное решение! У нас есть гарантия трудоустройства по договору, мы трудоустроили уже сотни студентов. У нас есть десятки компаний партнеров, которые просят нас подобрать им наших выпускников. У нас также есть стажировки в крупных компаниях, которые являются очень сильной строчкой в резюме наших выпускников. И у нас есть внутреннее AI HR агентство AI-hunter, которое специализируется только на трудоустройстве AI разработчиков. Вы точно сможете найти работу в качестве AI разработчика, обучаясь у нас.'

<h3><b> Диалог </h3>

In [ ]:
dialog = gpt.dialog()

С чем связан ваш интерес к искусственному интеллекту?
Клиент: Я хочу, чтобы после завершения обучения мне подобрали работу

Менеджер:  Конечно, мы можем помочь вам с трудоустройством. У нас есть гарантия трудоустройства по договору, мы уже трудоустроили сотни наших выпускников и у нас есть десятки
 компаний-партнеров, которые просят нас подобрать им AI разработчиков. Кроме того, у нас есть внутреннее AI HR агентство AI-hunter, которое специализируется только на
 трудоустройстве AI разработчиков. Мы можем гарантировать, что вы получите работу, связанную с AI-разработкой, после завершения обучения.

Отчёт системы:
 1. Потребность о трудоустройстве - была
2. Потребности о создании AI проекта - не было
3. Возражении о стоимости - не было
4. Возражении о времени обучения - не было

Менеджер:  "Отлично, что трудоустройство уже находится в списке ваших потребностей. А можете ли вы представить, как вы сможете использовать AI на своей будущей работе? Например,
 автоматизировать рутинные задачи 